<a href="https://colab.research.google.com/github/ekaadev/detection_comments/blob/main/detection_positif_and_negatif_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library & Dependency

In [1]:
# import dependency

# install gensim - pengolahan bahasa alami, scikit-learn(lib) - machine learning library
! pip install gensim scikit-learn -U
# add library

# gensim, library yang digunakan untuk pemrosesan baahsa alami(nlp)
import gensim
# logging, memberikan log / output pada saat program dijalankan
import logging
# import tagged document, untuk klasifikasikan dari sebuah kata yang telah diberi label
from gensim.models.doc2vec import TaggedDocument
# import model doc 2 vec
from gensim.models import Doc2Vec

# import random
import random
# import regex (regular expression)
import re
# import os untuk konfigurasi operating system
import os
# import numpy untuk pengolahan angka
import numpy as np

# import KNeighborsClassifier, untuk implementasi algoritma KNN
from sklearn.neighbors import KNeighborsClassifier
# import RandomForestClassifier, untuk implementasi algoritma RandomForest
from sklearn.ensemble import RandomForestClassifier
# import cross val predict, untuk generate estimasi nilai cross validated tiap input dari sebuah data
from sklearn.model_selection import cross_val_predict
# cosine similarity menghitung kemiripan dari 2 vektor
from sklearn.metrics.pairwise import cosine_similarity
# constructor pipeline dari input estimasi yang diberikan
from sklearn.pipeline import make_pipeline
# import CountVectorizer, untuk mengkovert koleksi dari sebuah text document ke dalam matrix, TfidfTransformer, menstransformasikan total matrix menjadi tf atau tfid yang normal (tfidf, cara menentukan kata itu penting atau tidak dalam suatu dokumen terhadap kumpulan dokumen menggunakan metode statistik)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


# Download dataset dan model pretrained

In [2]:
import kagglehub

# import data
# googlenewvectornegative300
# stanford sentiment treebank

# Download latest version
path = kagglehub.dataset_download("leadbest/googlenewsvectorsnegative300")
path2 = kagglehub.dataset_download("atulanandjha/stanford-sentiment-treebank-v2-sst2")

print("Path to dataset files:", path)
print("Path to dataset files:", path2)

100%|██████████| 19.1M/19.1M [00:00<00:00, 143MB/s]

Extracting files...


Path to dataset files: /kaggle/input/googlenewsvectorsnegative300
Path to dataset files: /root/.cache/kagglehub/datasets/atulanandjha/stanford-sentiment-treebank-v2-sst2/versions/30


# Memuat model pretained

In [3]:
# memuat models
gmodel = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)

# Membuat fungsi word extract

Fungsi ini digunakan untuk membersihkan sebuah kata agar kata yang dikumpulkan sudah sesuai dengan yang dibutuhkan

In [4]:
# fungsi regex
def word_extract(word):
  # lowercase
  word = word.lower()
  # menghilangkan tag html
  word = re.sub(r'<[^>]+>', ' ', word)
  # menghilangkan tanda petik diantara sebuah kata misal don't -> dont
  word = re.sub(r'(\w)\'(\w)', '\1\2', word)
  # mengganti selain alpha numerik menjadi spasi
  word = re.sub(r'\W', ' ', word)
  # mengganti spasi berlebihan menjadi satu spasi saja
  word = re.sub(r'\s+', ' ', word)
  # menghilangkan spasi diawal dan diakhir kata
  word = word.strip()
  # memisahkan kata menjadi per spasi
  return word.split()

# Menambahkan data

In [5]:
import csv

# unsupervised training data
unsup_sentences = []

# menambahkan data yang sudah diberikan tag ke dalam unsup_sentences, data tersebut berisi kata dari setiap file (imdb review, moview_review, stanford sentiment treebank)

!wget -P /tmp https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf /tmp/aclImdb_v1.tar.gz -C /tmp
for dirname in ['/tmp/aclImdb/train/neg', '/tmp/aclImdb/train/pos', '/tmp/aclImdb/test/neg', '/tmp/aclImdb/test/pos']:
  for filename in os.listdir(dirname):
    with open(os.path.join(dirname, filename), 'r') as f:
      words=word_extract(f.read())
      unsup_sentences.append(TaggedDocument(words, [dirname + '/' + filename]))

!wget -P /tmp http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
!tar -xzf /tmp/review_polarity.tar.gz -C /tmp
for dirname in ['/tmp/txt_sentoken/pos', '/tmp/txt_sentoken/neg']:
  for filename in os.listdir(dirname):
    with open(os.path.join(dirname, filename), 'r') as f:
      for i, sended in enumerate(f):
        words = word_extract(sended)
        unsup_sentences.append(TaggedDocument(words, ['%s/%s-%d' % (dirname, filename, i)]))

with open('/root/.cache/kagglehub/datasets/atulanandjha/stanford-sentiment-treebank-v2-sst2/versions/30/SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank/original_rt_snippets.txt', 'r', encoding='utf-8') as f:
    for i, sended in enumerate(f):
        words = word_extract(sended)
        unsup_sentences.append(TaggedDocument(words, [f'rt-{i}']))


--2025-05-10 16:24:00--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘/tmp/aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  67.3MB/s    in 1.2s    

2025-05-10 16:24:01 (67.3 MB/s) - ‘/tmp/aclImdb_v1.tar.gz’ saved [84125825/84125825]

--2025-05-10 16:24:21--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz [following]
--2025-05-10 16:24:21--  https://www.cs.cornell.edu/p

In [6]:
len(unsup_sentences)

125325

In [7]:
unsup_sentences[0:1]

[TaggedDocument(words=['once', 'upon', 'a', 'time', 'there', 'was', 'a', 'director', 'by', 'the', 'name', 'of', 'james', 'he', 'brought', 'us', 'wonderfully', 'thrilling', 'science', 'fiction', 'such', 'as', 'terminator', 'and', 'aliens', 'these', 'movies', 'were', 'the', 'stuff', 'blockbusters', 'were', 'made', 'of', 'and', 'he', 'looked', 'to', 'have', 'a', 'fantastic', 'future', 'ahead', 'of', 'him', 'as', 'the', 'dawn', 'of', 'computer', 'generated', 'special', 'effects', 'landed', 'upon', 'the', 'film', 'industry', 'terminator', '2', 'showed', 'gave', 'us', 'glimpses', 'of', 'what', 'was', 'possible', 'in', 'this', 'new', 'era', 'and', 'then', 'it', 'happened', '1997', 'countless', 'awards', 'obscene', 'amounts', 'of', 'money', 'outlandish', 'barrage', 'of', 'advertising', 'maximum', 'profit', 'margin', 'titanic', 'was', 'here', 'i', 'have', 'never', 'ever', 'been', 'one', 'to', 'jump', 'on', 'the', 'bandwagon', 'and', 'be', 'overly', 'critical', 'for', 'the', 'sake', 'of', 'it', 

# Class PermuteSentences

In [8]:
class PermuteSentences(object):
  # constructor
  def __init__(self, sentences):
    self.sentences=sentences

  def __iter__(self):
    shuffled = list(self.sentences)
    random.shuffle(shuffled)
    for sentence in shuffled:
      yield sentence

# Inisiliasi class PermuteSentences
Melakukan inisialiasi class permute sentences dan membuat model

In [9]:
permuter = PermuteSentences(unsup_sentences)
model = Doc2Vec(permuter, dm=0, hs=1, vector_size=50)

In [10]:
model.save('/content/reviews.d2v')

In [11]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip -O /tmp/sentiment_labelled_sentences.zip
!unzip /tmp/sentiment_labelled_sentences.zip -d /content/data

--2025-05-10 16:32:13--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘/tmp/sentiment_labelled_sentences.zip’

/tmp/sentiment_labe     [ <=>                ]  82.21K  --.-KB/s    in 0.02s   

2025-05-10 16:32:13 (3.23 MB/s) - ‘/tmp/sentiment_labelled_sentences.zip’ saved [84188]

Archive:  /tmp/sentiment_labelled_sentences.zip
   creating: /content/data/sentiment labelled sentences/
  inflating: /content/data/sentiment labelled sentences/.DS_Store  
   creating: /content/data/__MACOSX/
   creating: /content/data/__MACOSX/sentiment labelled sentences/
  inflating: /content/data/__MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: /content/data/sentiment labelled sentences/amazon_cells_l

# Testing model

In [18]:
# test model
sentences = []
sentvecs = []
sentiments = []
for filename in ['yelp', 'amazon_cells', 'imdb']:
  file_path = os.path.join('/content/data/sentiment labelled sentences', '%s_labelled.txt' % filename)
  with open(file_path, encoding='utf-8') as f:
    for i, line in enumerate(f):
      line_split = line.strip().split('\t')
      sentences.append(line_split[0])
      words = word_extract(line_split[0])
      sentvecs.append(model.infer_vector(words, epochs=10))
      sentiments.append(int(line_split[1]))

# shuffling
combined = list(zip(sentences, sentvecs, sentiments))
random.shuffle(combined)
sentences, sentvecs, sentiments = zip(*combined)

In [13]:
clf = KNeighborsClassifier(n_neighbors=9)
clfrf = RandomForestClassifier()

# Menghitung / Kalkulasi nilai rata-rata dan standard devisiasi

In [14]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(clf, sentvecs, sentiments, cv=5)
# menghitung mean(rata-rata)
# menghitung standard devisiasi
np.mean(score), np.std(score)

(0.782, 0.017461067804945055)

In [15]:
score = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
# menghitung mean(rata-rata)
# menghitung standard devisiasi
np.mean(score), np.std(score)

(0.7946666666666667, 0.01790096210946341)

In [16]:
pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), RandomForestClassifier())
score = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(score), np.std(score)

(0.7929999999999999, 0.013391539617733778)